[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ym001/Manteia/blob/master/notebook/notebook_Manteia_classification_cross_validation_run_in_colab.ipynb)


Run this nootbook with GPU : ->Mo[texte du lien](https://)difier->Parametre du nootbook->accélérateur matériel->GPU

In [1]:
pip install manteia


     |████████████████████████████████| 675kB 5.3MB/s 
     |████████████████████████████████| 3.8MB 8.5MB/s 
     |████████████████████████████████| 1.1MB 31.8MB/s 
     |████████████████████████████████| 890kB 37.7MB/s 
  Created wheel for manteia: filename=Manteia-0.0.35-cp36-none-any.whl size=33137 sha256=ac857cfc78ee236138e40003e73757c9bdd2174475b97f0cdff93cea42953fdb
  Stored in directory: /root/.cache/pip/wheels/6c/82/84/56e76c849dc788b0c53cf836f4bb4ed26d909063ce53c6c152
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=ed8a241133b1fe717b198e6cec2e8258f838ee2df249fedf0ca7f00025232a5d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=92c1bd5d44383e5e0b7e6c33051ccfae2127e48549758d7efb5c439c9d2961e5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f4

In [3]:
#modifier->parametre du notebook->GPU
#import nltk
#nltk.download('wordnet')
from Manteia.Classification import Classification 
from Manteia.Model import *
from Manteia.Dataset import Dataset
from Manteia.Preprocess import list_labels
from Manteia.Augmentation import *
from sklearn.model_selection import train_test_split,KFold

ds=Dataset('drugscom')
ds.documents_train=np.array(ds.documents_train[:100])
ds.labels_train=np.array(ds.labels_train[:100])

model = Model(model_name ='bert',early_stopping=True)
model.load_type()
model.load_tokenizer()
list_label=list_labels(ds.labels_train)
print(list_label)
model.num_labels=len(list_label)
model.load_class()
model.save('model_init')



#validation croisée
nb_pass=4
def coss_validation_idx(nb_pass,nb_docs):
  docs_idx = [idx for idx in range(nb_docs)]
  train_idx, test_idx = [], []
  for pli in range(nb_pass):
    test_pli_idx = list(np.random.choice(docs_idx,int(len(docs_idx)/nb_pass) , replace=False))
    train_pli_idx  = [idx for idx in docs_idx if idx not in test_pli_idx]
    train_idx.append(train_pli_idx)
    test_idx.append(test_pli_idx)
  return train_idx, test_idx
pli=0
acc = []
tr_idx,te_idx=coss_validation_idx(nb_pass,len(ds.documents_train))
for train_idx, test_idx in zip(tr_idx,te_idx):
  validation_idx = list(np.random.choice(train_idx,int(0.1*len(train_idx)) , replace=False))
  train_idx      = [idx for idx in train_idx if idx not in validation_idx]

  #doc_train_augmented,labels_train_augmented=eda(ds.documents_train[train_idx],ds.labels_train[train_idx])
  #doc_validation_augmented,labels_validation_augmented=eda(ds.documents_train[validation_idx],ds.labels_train[validation_idx])

  #doc_train_augmented,labels_train_augmented=uda(ds.documents_train[train_idx],ds.labels_train[train_idx])
  #doc_validation_augmented,labels_validation_augmented=uda(ds.documents_train[validation_idx],ds.labels_train[validation_idx])

  #doc_train_augmented,labels_train_augmented=pyramid(ds.documents_train[train_idx],ds.labels_train[train_idx])
  #doc_validation_augmented,labels_validation_augmented=pyramid(ds.documents_train[validation_idx],ds.labels_train[validation_idx])
	
  train_ids,train_masks           = encode_text(doc_train_augmented,model.tokenizer,model.MAX_SEQ_LEN)
  train_labels                    = encode_label(labels_train_augmented,list_label)

  validation_ids,validation_masks = encode_text(doc_validation_augmented,model.tokenizer,model.MAX_SEQ_LEN)
  validation_labels               = encode_label(labels_validation_augmented,list_label)

  test_ids,test_masks             = encode_text(ds.documents_train[test_idx],model.tokenizer,model.MAX_SEQ_LEN)
  test_labels                     = encode_label(ds.labels_train[test_idx],list_label)

  dt_train       = Create_DataLoader(train_ids,train_masks,train_labels)
  dt_validation  = Create_DataLoader(validation_ids,validation_masks,validation_labels)
  dt_test        = Create_DataLoader(test_ids,test_masks)
  
  model.load('model_init')

  model.configuration(dt_train)
  model.fit(dt_train,dt_validation)

  predictions=model.predict(dt_test,p_type='class')
  acc.append(accuracy(test_labels,predictions))
  print('Passe cross validation : {} , Accuracy : {}'.format(pli,acc[-1]))
  pli+=1

print('')
print('Cross validation accuracy : {} : {}'.format(np.mean(np.array(acc)),acc))


Loading bert tokenizer...
['1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0']
Loading bert class...


Token indices sequence length is longer than the specified maximum sequence length for this model (705 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


type compatible : bert-base-uncased
Loading bert class...
There are 1 GPU(s) available.
We will use the GPU: Tesla K80
There are 1 GPU(s) available.
We will use the GPU: Tesla K80

======== Epoch 1 / 20 ========
  Average training loss: 2.42

  Validation : Accuracy : 0.29
Validation accuracy increased (0.000000 --> 0.285714).  Saving model ...

======== Epoch 2 / 20 ========
  Average training loss: 2.05

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 3 / 20 ========
  Average training loss: 2.04

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 4 / 20 ========
  Average training loss: 1.95

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 5 / 20 ========
  Average training loss: 1.94

  Validation : Ac

Token indices sequence length is longer than the specified maximum sequence length for this model (705 > 512). Running this sequence through the model will result in indexing errors


Passe cross validation : 0 , Accuracy : 0.32


...

type compatible : bert-base-uncased
Loading bert class...
There are 1 GPU(s) available.
We will use the GPU: Tesla K80

======== Epoch 1 / 20 ========
  Average training loss: 2.26

  Validation : Accuracy : 0.29
Validation accuracy increased (0.000000 --> 0.285714).  Saving model ...

======== Epoch 2 / 20 ========
  Average training loss: 2.19

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 3 / 20 ========
  Average training loss: 2.11

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 4 / 20 ========
  Average training loss: 1.98

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 5 / 20 ========
  Average training loss: 1.91

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy inc

Token indices sequence length is longer than the specified maximum sequence length for this model (705 > 512). Running this sequence through the model will result in indexing errors


Passe cross validation : 1 , Accuracy : 0.28
type compatible : bert-base-uncased
Loading bert class...
There are 1 GPU(s) available.
We will use the GPU: Tesla K80

======== Epoch 1 / 20 ========
  Average training loss: 2.39

  Validation : Accuracy : 0.29
Validation accuracy increased (0.000000 --> 0.285714).  Saving model ...

======== Epoch 2 / 20 ========
  Average training loss: 2.19

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 3 / 20 ========
  Average training loss: 2.07

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 4 / 20 ========
  Average training loss: 2.04

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 5 / 20 ========
  Average training loss: 1.95

  Validation : Accuracy : 0.29
Sa

Token indices sequence length is longer than the specified maximum sequence length for this model (705 > 512). Running this sequence through the model will result in indexing errors


Passe cross validation : 2 , Accuracy : 0.24
type compatible : bert-base-uncased
Loading bert class...
There are 1 GPU(s) available.
We will use the GPU: Tesla K80

======== Epoch 1 / 20 ========
  Average training loss: 2.32

  Validation : Accuracy : 0.29
Validation accuracy increased (0.000000 --> 0.285714).  Saving model ...

======== Epoch 2 / 20 ========
  Average training loss: 2.15

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 3 / 20 ========
  Average training loss: 2.05

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 4 / 20 ========
  Average training loss: 1.99

  Validation : Accuracy : 0.29
Save model : 0 out of 2
Validation accuracy increased (0.285714 --> 0.285714).  Saving model ...

======== Epoch 5 / 20 ========
  Average training loss: 1.88

  Validation : Accuracy : 0.29
Sa